# Transformers 설치

In [ ]:
!pip install transformers

     |████████████████████████████████| 890kB 2.4MB/s 
     |████████████████████████████████| 890kB 13.1MB/s 
     |████████████████████████████████| 1.1MB 15.3MB/s 
     |████████████████████████████████| 3.0MB 21.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4e02ef2cc1801f468d948b41101081d2029a4ace13442a9acfb0cf2f52028e4e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## 1) 간결하고 사용성이 좋은 API

In [ ]:
from transformers import AutoTokenizer, AutoModel

pretrained_model = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
model = AutoModel.from_pretrained(pretrained_model)

### 1.1 Tokenizer


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
raw_text = '토크나이저의 결과입니다.'
input_ids = tokenizer(raw_text)['input_ids']
print(input_ids)

[101, 9873, 20308, 16439, 10739, 48387, 10459, 85533, 58303, 48345, 119, 102]


### 1.2 Transformers Model

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = AutoModel.from_pretrained('bert-base-multilingual-cased')

inputs = tokenizer('토크나이저의 결과입니다.', return_tensors='pt')
outputs = model(**inputs)[0]
print(outputs)

tensor([[[-0.0493, -0.1934, -0.4559,  ...,  0.2070,  0.1388,  0.1115],
         [-0.4179, -0.6123, -0.2678,  ...,  0.3105,  0.0795, -0.2036],
         [ 0.1398, -0.6489,  0.3444,  ...,  0.2988,  0.2643, -0.0940],
         ...,
         [-0.2535, -0.3039,  0.2575,  ..., -0.0936,  0.2240,  0.2749],
         [-0.1312, -0.3211,  0.4351,  ...,  0.3288, -0.1510,  0.1887],
         [-0.2148, -0.3251,  0.0978,  ...,  0.1261,  0.2307,  0.1305]]],
       grad_fn=<NativeLayerNormBackward>)


### 1.3 Heads

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

#### (별첨) Customized head

In [ ]:
import torch
from transformers import BertPreTrainedModel, BertConfig, BertModel

class BertForCustomClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.transformer_model = BertModel(config)

        # customized output layer 정의
        self.custom_layer = torch.nn.Linear(config.hidden_size, config.output_size)
        self.init_weights()

    def forward(self, **kwargs):
        contextual_embedded_vector = self.transformer_model(**kwargs)[1]
        logits = self.custom_layer(contextual_embedded_vector)

        # loss와 함께 정의
        loss = torch.nn.CrossEntropyLoss(logits, labels)
        return ((loss,) + contextual_embedded_vector)

config = BertConfig.from_pretrained('bert-base-multilingual-cased')
config.output_size = 2
model = BertForCustomClassification.from_pretrained('bert-base-multilingual-cased', config=config)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForCustomClassification: ['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.weight', 'bert.encoder.layer.0.output.dense.bias', 'bert.encoder.layer.1.attention.self.query.weight', 'bert.encoder.layer.1.attention.self.query.bias', 'bert.encoder.layer.1.attention.self.ke

## 2) Pytorch와 Tensorflow간 높은 호환성

In [ ]:
from transformers import BertTokenizer, TFBertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained('bert-base-multilingual-cased', from_pt=True)

inputs = tokenizer('토크나이저의 결과입니다.', return_tensors='tf')
outputs = model(inputs)[0]
print(outputs)

All PyTorch model weights were used when initializing TFBertModel.

Some weights or buffers of the PyTorch model TFBertModel were not initialized from the TF 2.0 model and are newly initialized: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tf.Tensor(
[[[-0.04932138 -0.1934443  -0.45592806 ...  0.20699202  0.13877389
    0.1114625 ]
  [-0.41787338 -0.6123404  -0.26776335 ...  0.31048116  0.0795071
   -0.20364681]
  [ 0.13984454 -0.6488731   0.34438094 ...  0.29884517  0.26428992
   -0.09395621]
  ...
  [-0.25347716 -0.30392265  0.25751013 ... -0.09357093  0.22395872
    0.27494037]
  [-0.1311921  -0.32111168  0.43514502 ...  0.32883495 -0.15097503
    0.18871985]
  [-0.21476766 -0.3250965   0.09775992 ...  0.12612699  0.2306703
    0.13046268]]], shape=(1, 12, 768), dtype=float32)
